__Zarr Development Notebooks__</br>
<img src="http://static1.squarespace.com/static/530979d9e4b04bff4a3aadf5/t/5446c34ce4b0e5a2c7ff2614/1413923661373/DewberryLogo_RGB.png?format=1500w" width="40%" align='right'/>
__Probability Risk: Post Processor Group__</br>
PYTHON 3.6</br>
Overview: This notebook is to analyze how to store WSE tifs from multiple models into one cube of data.</br>
Updated 2018-10-15</br>
by Stephen Duncan: sduncan@dewberry.com <br/>

*Use Environment with boto3, zarr, & s3fs*

In [1]:
import os, sys, time, glob, s3fs, zarr, json, boto3
import pathlib as pl
import numpy as np
import pandas as pd
from io import BytesIO
from osgeo import gdal, osr
from matplotlib import pyplot as plt
from IPython.display import display, Markdown, Latex
%matplotlib inline

In [2]:
def get_files(bucketname, prefixname, textstring):
    '''Gets Files list from AWS based on a bucket name and prefix'''
    wselist = []
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(name=bucketname)
    FilesNotFound = True
    for obj in bucket.objects.filter(Prefix=prefixname):
        if textstring in str(obj) and 'xml' not in str(obj):
            #print('{0}:{1}'.format(bucket.name, obj.key))
            wselist.append('{0}:{1}'.format(bucket.name, obj.key))
        FilesNotFound = False
    if FilesNotFound:
        print("ALERT", "No file in {0}/{1}".format(bucket, prefixname))
    return sorted(wselist)

def mprint(txt:str) -> print:
    '''Print Text as Markdown for Jupyter Lab and Notebooks'''
    display(Markdown(txt))

# Get Tif Locations

In [4]:
bucket = 'probmodelingrepository'
prefix = 'AR-LA-MS-River/ProductionRuns/outputs/BaseMiss' #'Augusta-Levee-AOP/ProductionRuns/outputs/Pluvial'
searchfor = 'WSE_'

s3tifs = get_files(bucket,prefix,searchfor)
mprint(f'## Tifs Found: {len(s3tifs)}')
mprint('The first couple are located at:')
mprint('- '+'\n- '.join(s3tifs[0:5]))

## Tifs Found: 300

The first couple are located at:

- probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E301/WSE_BaseMiss_E301.tif
- probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E302/WSE_BaseMiss_E302.tif
- probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E303/WSE_BaseMiss_E303.tif
- probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E304/WSE_BaseMiss_E304.tif
- probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E305/WSE_BaseMiss_E305.tif

In [15]:
zeropaddedstr = '_E{:03d}.tif'
lost_tifs = []
for i in range(1200):
    j = i+1
    notfound = True
    for tif in s3tifs:
        if tif.find(zeropaddedstr.format(j)) >= 0:
            notfound = False
    if notfound:
        lost_tifs.append('WSE_Pluvial'+zeropaddedstr.format(j))
mprint(f'## Tifs Lost: {len(lost_tifs)}')
mprint(''+'\n- '.join(lost_tifs))

## Tifs Lost: 198

WSE_Pluvial_E095.tif
- WSE_Pluvial_E096.tif
- WSE_Pluvial_E097.tif
- WSE_Pluvial_E098.tif
- WSE_Pluvial_E099.tif
- WSE_Pluvial_E100.tif
- WSE_Pluvial_E101.tif
- WSE_Pluvial_E102.tif
- WSE_Pluvial_E103.tif
- WSE_Pluvial_E104.tif
- WSE_Pluvial_E105.tif
- WSE_Pluvial_E106.tif
- WSE_Pluvial_E107.tif
- WSE_Pluvial_E108.tif
- WSE_Pluvial_E109.tif
- WSE_Pluvial_E110.tif
- WSE_Pluvial_E111.tif
- WSE_Pluvial_E112.tif
- WSE_Pluvial_E113.tif
- WSE_Pluvial_E114.tif
- WSE_Pluvial_E115.tif
- WSE_Pluvial_E116.tif
- WSE_Pluvial_E117.tif
- WSE_Pluvial_E118.tif
- WSE_Pluvial_E119.tif
- WSE_Pluvial_E120.tif
- WSE_Pluvial_E121.tif
- WSE_Pluvial_E122.tif
- WSE_Pluvial_E123.tif
- WSE_Pluvial_E124.tif
- WSE_Pluvial_E125.tif
- WSE_Pluvial_E126.tif
- WSE_Pluvial_E127.tif
- WSE_Pluvial_E128.tif
- WSE_Pluvial_E129.tif
- WSE_Pluvial_E130.tif
- WSE_Pluvial_E131.tif
- WSE_Pluvial_E132.tif
- WSE_Pluvial_E133.tif
- WSE_Pluvial_E134.tif
- WSE_Pluvial_E135.tif
- WSE_Pluvial_E136.tif
- WSE_Pluvial_E137.tif
- WSE_Pluvial_E138.tif
- WSE_Pluvial_E139.tif
- WSE_Pluvial_E140.tif
- WSE_Pluvial_E141.tif
- WSE_Pluvial_E142.tif
- WSE_Pluvial_E143.tif
- WSE_Pluvial_E144.tif
- WSE_Pluvial_E145.tif
- WSE_Pluvial_E146.tif
- WSE_Pluvial_E147.tif
- WSE_Pluvial_E148.tif
- WSE_Pluvial_E149.tif
- WSE_Pluvial_E150.tif
- WSE_Pluvial_E151.tif
- WSE_Pluvial_E152.tif
- WSE_Pluvial_E153.tif
- WSE_Pluvial_E154.tif
- WSE_Pluvial_E155.tif
- WSE_Pluvial_E156.tif
- WSE_Pluvial_E157.tif
- WSE_Pluvial_E158.tif
- WSE_Pluvial_E159.tif
- WSE_Pluvial_E160.tif
- WSE_Pluvial_E161.tif
- WSE_Pluvial_E162.tif
- WSE_Pluvial_E163.tif
- WSE_Pluvial_E164.tif
- WSE_Pluvial_E165.tif
- WSE_Pluvial_E166.tif
- WSE_Pluvial_E167.tif
- WSE_Pluvial_E168.tif
- WSE_Pluvial_E169.tif
- WSE_Pluvial_E170.tif
- WSE_Pluvial_E171.tif
- WSE_Pluvial_E172.tif
- WSE_Pluvial_E173.tif
- WSE_Pluvial_E174.tif
- WSE_Pluvial_E175.tif
- WSE_Pluvial_E176.tif
- WSE_Pluvial_E177.tif
- WSE_Pluvial_E178.tif
- WSE_Pluvial_E179.tif
- WSE_Pluvial_E180.tif
- WSE_Pluvial_E181.tif
- WSE_Pluvial_E182.tif
- WSE_Pluvial_E183.tif
- WSE_Pluvial_E184.tif
- WSE_Pluvial_E185.tif
- WSE_Pluvial_E186.tif
- WSE_Pluvial_E187.tif
- WSE_Pluvial_E188.tif
- WSE_Pluvial_E189.tif
- WSE_Pluvial_E190.tif
- WSE_Pluvial_E191.tif
- WSE_Pluvial_E192.tif
- WSE_Pluvial_E193.tif
- WSE_Pluvial_E194.tif
- WSE_Pluvial_E195.tif
- WSE_Pluvial_E196.tif
- WSE_Pluvial_E197.tif
- WSE_Pluvial_E198.tif
- WSE_Pluvial_E199.tif
- WSE_Pluvial_E200.tif
- WSE_Pluvial_E201.tif
- WSE_Pluvial_E202.tif
- WSE_Pluvial_E203.tif
- WSE_Pluvial_E204.tif
- WSE_Pluvial_E205.tif
- WSE_Pluvial_E206.tif
- WSE_Pluvial_E207.tif
- WSE_Pluvial_E208.tif
- WSE_Pluvial_E209.tif
- WSE_Pluvial_E210.tif
- WSE_Pluvial_E211.tif
- WSE_Pluvial_E212.tif
- WSE_Pluvial_E213.tif
- WSE_Pluvial_E214.tif
- WSE_Pluvial_E215.tif
- WSE_Pluvial_E216.tif
- WSE_Pluvial_E217.tif
- WSE_Pluvial_E218.tif
- WSE_Pluvial_E219.tif
- WSE_Pluvial_E220.tif
- WSE_Pluvial_E221.tif
- WSE_Pluvial_E222.tif
- WSE_Pluvial_E223.tif
- WSE_Pluvial_E224.tif
- WSE_Pluvial_E225.tif
- WSE_Pluvial_E226.tif
- WSE_Pluvial_E227.tif
- WSE_Pluvial_E228.tif
- WSE_Pluvial_E229.tif
- WSE_Pluvial_E230.tif
- WSE_Pluvial_E231.tif
- WSE_Pluvial_E232.tif
- WSE_Pluvial_E233.tif
- WSE_Pluvial_E234.tif
- WSE_Pluvial_E235.tif
- WSE_Pluvial_E236.tif
- WSE_Pluvial_E237.tif
- WSE_Pluvial_E238.tif
- WSE_Pluvial_E239.tif
- WSE_Pluvial_E240.tif
- WSE_Pluvial_E241.tif
- WSE_Pluvial_E242.tif
- WSE_Pluvial_E243.tif
- WSE_Pluvial_E244.tif
- WSE_Pluvial_E245.tif
- WSE_Pluvial_E246.tif
- WSE_Pluvial_E247.tif
- WSE_Pluvial_E248.tif
- WSE_Pluvial_E249.tif
- WSE_Pluvial_E250.tif
- WSE_Pluvial_E252.tif
- WSE_Pluvial_E253.tif
- WSE_Pluvial_E254.tif
- WSE_Pluvial_E255.tif
- WSE_Pluvial_E256.tif
- WSE_Pluvial_E257.tif
- WSE_Pluvial_E258.tif
- WSE_Pluvial_E259.tif
- WSE_Pluvial_E260.tif
- WSE_Pluvial_E261.tif
- WSE_Pluvial_E262.tif
- WSE_Pluvial_E263.tif
- WSE_Pluvial_E264.tif
- WSE_Pluvial_E265.tif
- WSE_Pluvial_E266.tif
- WSE_Pluvial_E267.tif
- WSE_Pluvial_E268.tif
- WSE_Pluvial_E269.tif
- WSE_Pluvial_E270.tif
- WSE_Pluvial_E271.tif
- WSE_Pluvial_E272.tif
- WSE_Pluvial_E273.tif
- WSE_Pluvial_E274.tif
- WSE_Pluvial_E275.tif
- WSE_Pluvial_E276.tif
- WSE_Pluvial_E277.tif
- WSE_Pluvial_E278.tif
- WSE_Pluvial_E279.tif
- WSE_Pluvial_E280.tif
- WSE_Pluvial_E281.tif
- WSE_Pluvial_E282.tif
- WSE_Pluvial_E283.tif
- WSE_Pluvial_E284.tif
- WSE_Pluvial_E285.tif
- WSE_Pluvial_E286.tif
- WSE_Pluvial_E287.tif
- WSE_Pluvial_E288.tif
- WSE_Pluvial_E289.tif
- WSE_Pluvial_E290.tif
- WSE_Pluvial_E291.tif
- WSE_Pluvial_E881.tif
- WSE_Pluvial_E1112.tif